<h1><center>Segmenting and Clustering Neighborhoods in Toronto</center></h1>
<h3><center>Problem 1</center></h3>
<h3><center>Scraping Data from Wikipedia</center></h3>

In [188]:
# Problem 1

# Scraping Data from Wikipedia

<h4>The URL to the page on Wikipedia</h4>

In [17]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

<h4>Import Libraries</h4>

In [229]:
# Imports for Problem 1
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# Imports for Problem 2

# Imports for Problem 3
import numpy as np
from sklearn.cluster import KMeans
# !pip install folium
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

<h4>Request page from URL</h4>

In [19]:
request = requests.get(URL)
request = request.text

<h4>Create a Beautiful Soup Object and extract tables</h4>

In [20]:
soup = BeautifulSoup(request, 'xml')
table = soup.find('table')
print(f'Type : {type(table)}')
print(f'Length : {len(table)}')

Type : <class 'bs4.element.Tag'>
Length : 2


<h4>Extract all the data from the table into python lists</h4>

In [145]:
# Search all the postcode, borough, neighborhood 
PostalCode = []
Borough = []
Neighborhood = []
for tr in table.find_all('tr'):
    for b in tr.find_all('b'):
        PostalCode.append(b.text)
    for span in tr.find_all('span'):
        span = span.text
        borough = ""
        neigh = re.search('\((.*?)\)', span)
        if neigh is not None:
            neigh = neigh.group()
            borough = span.replace(neigh,"")
            neigh = neigh.replace("/", ",")
            brackets = ('()') 
            for x in neigh:
                if x in brackets:
                    neigh = neigh.replace(x, "") 
        else:
            neigh = 'Not Assigned'
        if borough is "":
            borough = 'Not Assigned'
        Borough.append(borough)
        Neighborhood.append(neigh)
        

print(f'Length of PostalCode : {len(PostalCode)}')
print(f'Length of Neighborhood : {len(Neighborhood)}')
print(f'Length of Borough : {len(Borough)}')

Length of PostalCode : 180
Length of Neighborhood : 180
Length of Borough : 180


<h4>Create a pandas Dataframe</h4>

In [146]:
df = pd.DataFrame({
    'PostalCode' : PostalCode,
    'Borough' : Borough,
    'Neighborhood' : Neighborhood
})

<h4>Display first 5 rows of dataframe</h4>

In [140]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not Assigned,Not Assigned
1,M2A,Not Assigned,Not Assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


<h4>Remove all rows with Borough is 'Not Assigned'</h4>

In [141]:
pbn_df = df[df['Borough'] != 'Not Assigned']

<h4>Reset Index</h4>

In [147]:
pbn_df = pbn_df.reset_index(drop=True)
pbn_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


<h4>Display Shape</h4>

In [144]:
pbn_df.shape

(103, 3)

<h3><center>Problem 2</center></h3>
<h3><center>Getting Latitude and Longitude</center></h3>

In [189]:
# Problem 2

# Getting Latitude and Longitude

<h4>Import Libraries</h4>

In [157]:
# All import are performed at one place on the top of the notebook

In [172]:
pll_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [174]:
pll_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
pll_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [177]:
pbnll_df = pd.merge(pbn_df, pll_df, on='PostalCode')

In [191]:
pbnll_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<h3><center>Problem 3</center></h3>
<h3><center>Exploring Toronto Borough</center></h3>

In [194]:
# Problem 3

# Exploring Toronto Borough

In [197]:
toronto_df = pbnll_df[pbnll_df['Borough'].str.contains('Toronto')]
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [199]:
# The code was removed by Watson Studio for sharing.

In [204]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    
    for name, latitude, longitude in zip(names, latitudes, longitudes):
        # create URL
        URL = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={latitude},{longitude}&radius={RADIUS}&limit={LIMIT}'
        
        # make get request
        request = requests.get(URL).json()["response"]['groups'][0]['items']
        
        # get data
        venues_list.append([(
            name, 
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in request])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [205]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

In [207]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [209]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,61,61,61,61,61,61
"Brockton , Parkdale Village , Exhibition Place",23,23,23,23,23,23
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",13,13,13,13,13,13
Central Bay Street,63,63,63,63,63,63
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35
Davisville North,7,7,7,7,7,7


In [212]:
# One Hot Encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(1579, 232)

In [213]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.016393,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,"CN Tower , King and Spadina , Railway Lands , ...",0.0,0.0,0.076923,0.076923,0.153846,0.230769,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
3,Central Bay Street,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.015873,0.0,0.0,0.015873,0.0,0.0,0.015873
4,Christie,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000


In [214]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [217]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Seafood Restaurant,Restaurant,Pharmacy,Beer Bar,Diner
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Convenience Store,Stadium,Restaurant,Italian Restaurant,Intersection,Bar
2,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Harbor / Marina,Boat or Ferry,Rental Car Location,Sculpture Garden,Boutique,Plane,Airport Food Court,Airport
3,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bubble Tea Shop,Thai Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Salad Place
4,Christie,Grocery Store,Café,Park,Nightclub,Baby Store,Italian Restaurant,Restaurant,Athletics & Sports,Candy Store,Coffee Shop


In [223]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int32)

In [225]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Breakfast Spot,Pub,Theater,Café,Mexican Restaurant,Bank,Event Space
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Italian Restaurant,Café,Japanese Restaurant,Hotel,Middle Eastern Restaurant,Bubble Tea Shop,Cosmetics Shop,Movie Theater
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Cocktail Bar,Gastropub,Department Store,Cosmetics Shop,Italian Restaurant,Cheese Shop,Park,Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Pub,Health Food Store,Coffee Shop,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Seafood Restaurant,Restaurant,Pharmacy,Beer Bar,Diner


In [226]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Seafood Restaurant,Restaurant,Pharmacy,Beer Bar,Diner
1,1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Convenience Store,Stadium,Restaurant,Italian Restaurant,Intersection,Bar
2,1,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Harbor / Marina,Boat or Ferry,Rental Car Location,Sculpture Garden,Boutique,Plane,Airport Food Court,Airport
3,1,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bubble Tea Shop,Thai Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Salad Place
4,1,Christie,Grocery Store,Café,Park,Nightclub,Baby Store,Italian Restaurant,Restaurant,Athletics & Sports,Candy Store,Coffee Shop


In [230]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


In [231]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters